<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# The `ghl_calculate_phitilde_rhs` Function

## Authors: Samuel Cupp, Leo Werneck, and Zach Etienne

<font color='red'>**This module is currently under development**</font>

### Required and recommended citations:
* **(Required)** GRHayL (TBD)
* **(Required)** [Etienne, Paschalidis, Haas, Mösta, and Shapiro. IllinoisGRMHD: an open-source, user-friendly GRMHD code for dynamical spacetimes (2015)](http://arxiv.org/abs/1501.07276).

<a id='introduction'></a>

# Introduction \[Back to [top](#toc)\]
$$\label{introduction}$$

The `ghl_calculate_phitilde_rhs` function $\tilde{\Phi}_\mathrm{RHS}$, including the generalized Lorenz gauge contribution. This function expects the precomputation of intermediate quantities, which is provided by the `ghl_interpolate_for_induction_rhs` function. This separation allows for OMP parallelism, as the second function needs stencils of the intermediate quantities computed by `ghl_interpolate_for_induction_rhs`.

The gauge contributions to $A_i^\mathrm{RHS}$ are not included in this function because they are computed with finite difference stencils of output from `ghl_interpolate_for_induction_rhs`.

<a id='arglist'></a>

## Function Argument List
$$\label{arglist}$$

The prototype for the `ghl_calculate_phitilde_rhs` function is
```
double ghl_calculate_phitilde_rhs(
      const double dxi[3],
      const double Lorenz_damping_factor,
      const double alpha_interp,
      const double shiftx_interp[5],
      const double shifty_interp[5],
      const double shiftz_interp[5],
      const double Ai_stencil[3][2],
      const double phitilde_stencil[3][5])
```

`dxi` is an array of the inverse grid spacing
$$
\mathrm{dxi[3]} = \{\frac{1}{dx}, \frac{1}{dy}, \frac{1}{dz}\}
$$
`Lorenz_damping_factor` sets the damping factor for the generalized Lorenz gauge term. `shiftx_interp`, `shifty_interp`, and `shiftz_interp` are stencils of the interpolated shift ranging from $i-2$ to $i+2$ in the direction of the component (e.g. shiftx_interp is a 1D stencil in the x direction). `phitilde_stencil` provides an identical stencil, but it is needed for all three directions. Hence, the first index represents the coordinate and the second represents the stencil. Finally, `Ai_stencil` contains the `alpha_sqrtg_A*_interp` stencils provided by `ghl_interpolate_for_induction_rhs`.

<a id='inputs'></a>

# Function Internals
$$\label{inputs}$$

$\tilde{\Phi}_\mathrm{RHS}$ is given by
$$
\tilde{\Phi}_\mathrm{RHS} = \mathrm{[shift\ advection\ term]} + \partial_j (\alpha \sqrt{\gamma} A^j)
$$
This function first computes the advection term. The exact computation depends on the value of the shift at the point of interest. If the shift is negative, then the term is a stencil from $i-2$ to $i$. Otherwise, it is a stencil from $i$ to $+2$. If we consider the $x$ direction, then for negative $\beta^x$, the RHS is given by
$$
\tilde{\Phi}_{\mathrm{RHS},\beta^x} = \frac{1}{\Delta x}\left( \beta^x_{i-2}\tilde{\Phi}_{i-2} -4\beta^x_{i-1}\tilde{\Phi}_{i-1} + 3\beta^x_{i}\tilde{\Phi}_{i}\right)
$$

For positive $\beta^x$, it is instead
$$
\tilde{\Phi}_{\mathrm{RHS},\beta^x} = -\frac{1}{\Delta x}\left( \beta^x_{i+2}\tilde{\Phi}_{i+2} -4\beta^x_{i+1}\tilde{\Phi}_{i+1} + 3\beta^x_{i}\tilde{\Phi}_{i}\right)
$$

The contributions from $\beta^y$ and $\beta^z$ are identical, but with the stencils in $j$ and $k$, respectively. After these, we add the
$$
\partial_j (\alpha \sqrt{\gamma} A^j)
$$
term by using the `Ai_stencil` array. Finally, we compute the damping factor by adding
$$
- (\mathrm{Lorenz\_damping\_factor})\ \alpha \tilde{\Phi}
$$